In [1]:
import sys
!python3 -m spacy download en
import re
import spacy
import pandas as pd

import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# from gensim.summarization.summarizer import summarize
from summa.summarizer import summarize

# ps=SnowballStemmer('english')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words = set(stop_words)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/kamal/q_3.7/lib/python3.7/site-packages/en_core_web_sm -->
/home/kamal/q_3.7/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
s = './summaries.csv'
f = './fulltext.csv'

# s = sys.argv[1]
# f = sys.argv[2]

ds = pd.read_csv(s)['abstract'].values.tolist()
df = pd.read_csv(f)['paper text'].values.tolist()

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def preprocessing(data):  # data is a list of sentences/abstracts
    data = [re.sub('\s+', ' ', sent) for sent in data]
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases. 
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words, stop_words)
    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
    nlp = spacy.load('en', disable=['parser', 'ner'])
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized

In [4]:
ds = preprocessing(ds)
df = preprocessing(df)

In [22]:
# new_df = []
# for i in range(len(df)):
# #     print(df[i])
#     x = '_'.join(df[i])
#     x = summarize(x,words=500)
#     print(x)
#     new_df.append(x)

In [ ]:
arr = ds + df

In [ ]:
# # Remove stop words and punctuation followed by stemming
# def foo_filter(sentence):
#     sentence=sentence.lower()
#     word_tokens=word_tokenize(sentence)
#     word_tokens = [w.lower() for w in word_tokens if not (w in stop_words or w in string.punctuation)]
# #     filtered_sentence = [ps.stem(w) for w in word_tokens]
#     return word_tokens
# #     return filtered_sentence

In [ ]:
# filter("This is a sample sentence, showing off the stop words filtration 10 .")
# filter_sent=[]
# for i in data1:
#     filter_sent.append(foo_filter(i))

In [ ]:
# import tensorflow as tf
# import tensorflow_hub as hub
# tf.logging.set_verbosity(tf.logging.ERROR)
# embed = hub.Module("./sentence_wise_email/module/module_useT")
# sent_messages = [' '.join(i) for i in filter_sent]# Reduce logging output.
# just_sent_messages = [' '.join(i) for i in filter_just_sent]# Reduce logging output.
# with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     sent_embeddings = session.run(embed(sent_messages))
#     just_sent_embeddings = session.run(embed(just_sent_messages))
# print(sent_embeddings[0],just_sent_embeddings[0])

In [ ]:
# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('./dataset/GoogleNews-vectors-negative300.bin', binary=True)
from gensim.models import Word2Vec
from fse.models import Average
# from fse.models import Sentence2Vec
# from gensim.models.doc2vec import Doc2Vec

max_epochs = 100
vec_size = 20
alpha = 0.025

# model = Doc2Vec(size=vec_size,
#                 alpha=alpha, 
#                 min_alpha=0.00025,
#                 min_count=1,
#                 dm =1)


model = Word2Vec(arr, min_count=0)
se = Average(model)
# sentences_emb = se.train(filter_sent)

In [ ]:
givenSent="We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset."
filter_sent=foo_filter(givenSent)
# given_emb=se.train(filter_sent)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# max_i=-1
# max_v=-1e9
# for i in range(len(sentences_emb)):
#     c=cosine_similarity([sentences_emb[i]],[given_emb[0]])
#     if c[0][0] > max_v:
#         max_v=c[0][0]
#         max_i=i
# print(max_i,data1[max_i])

In [ ]:
from math import exp
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity(score):
    score*=1.5
    if score > 1:
        score=1
    if score < -1:
        score=-1
    score=(score+1)/2
    if score < 0.5:
        score*=0.75
    return score

# f_s0=foo_filter(s0)
# f_s1=foo_filter(s1)
# f_s3=foo_filter(s3)
# f_s5=foo_filter(s5)
# f_s7=foo_filter(s7)
# # print(' '.join(foo_filter(s2)))

# s2=' '.join(foo_filter(s2))
# s4=' '.join(foo_filter(s4))
# s6=' '.join(foo_filter(s6))
# s8=' '.join(foo_filter(s8))
# # print(s2,'\n\n',s4)
# s2=summarizer.summarize(s2,words=500)
# s4=summarizer.summarize(s4,words=500)
# s6=summarizer.summarize(s6,words=len(s1.split(' ')))
# s8=summarizer.summarize(s8,words=len(s1.split(' ')))
# # print(s4)
# # print(s1,'\n\n\n',s2)
# f_s2=foo_filter(s2)
# f_s4=foo_filter(s4)
# f_s6=foo_filter(s8)
# f_s8=foo_filter(s8)
# # sentences=[f_s0,f_s1,f_s2,f_s3,f_s4]
# sentences=[f_s0,f_s1,f_s2,f_s3,f_s4,f_s5,f_s6,f_s7,f_s8]
# emb=se.train(sentences)
# print(emb.shape)
# emb_f_s0=[emb[0,:]]
# emb_f_s1=[emb[1,:]]
# emb_f_s2=[emb[2,:]]
# emb_f_s3=[emb[3,:]]
# emb_f_s4=[emb[4,:]]
# emb_f_s5=[emb[5,:]]
# emb_f_s6=[emb[6,:]]
# emb_f_s7=[emb[7,:]]
# emb_f_s8=[emb[8,:]]


# c02=cosine_similarity(emb_f_s0,emb_f_s2)
# c12=cosine_similarity(emb_f_s1,emb_f_s2)
# c32=cosine_similarity(emb_f_s3,emb_f_s2)
# c52=cosine_similarity(emb_f_s5,emb_f_s2)
# c72=cosine_similarity(emb_f_s7,emb_f_s2)

# c04=cosine_similarity(emb_f_s0,emb_f_s4)
# c14=cosine_similarity(emb_f_s1,emb_f_s4)
# c34=cosine_similarity(emb_f_s3,emb_f_s4)
# c54=cosine_similarity(emb_f_s5,emb_f_s4)
# c74=cosine_similarity(emb_f_s7,emb_f_s4)

# # print(get_similarity(c02[0][0]),get_similarity(c12[0][0]),get_similarity(c32[0][0]))
# print(get_similarity(c02[0][0]),get_similarity(c12[0][0]),get_similarity(c32[0][0]),get_similarity(c52[0][0]),get_similarity(c72[0][0]))
# # print(get_similarity(c04[0][0]),get_similarity(c14[0][0]),get_similarity(c34[0][0]))
# print(get_similarity(c04[0][0]),get_similarity(c14[0][0]),get_similarity(c34[0][0]),get_similarity(c54[0][0]),get_similarity(c74[0][0]))
